In [2]:
import time
import math
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from lightfm.evaluation import precision_at_k

from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse as sp
from skopt import forest_minimize

from lightfm import LightFM

import warnings
warnings.filterwarnings('ignore')

In [3]:
SEED = 42

### Коллаборативный подход

In [4]:
df = pd.read_csv('rating.csv')

Для ускорения процесса возьмем первые 100к записей.

In [5]:
ratings = df[:100000].drop(columns=['timestamp'])

In [6]:
del df

In [7]:
ratings.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


Посмотрим на минимальное количество голосов, приходящихся на одного пользователя в датафрейме:

In [8]:
ratings.groupby('userId').size().min()

20

Минимальное количество голосов пользователя =20, чего нам хватит для рекомендации фильмов и оценки качества наших рекомендаций.

Для разбиения на train/test будем использовать Stratified Split, чтобы оценки каждого пользователя, представленного в тренировочном наборе данных, были также и в тестовом наборе в одинаковом для всех пользователей отношении, равном 3:1.

In [9]:
train_data, test_data = train_test_split(ratings,
                                         test_size=0.25,
                                         stratify=ratings['userId'],
                                         random_state=SEED)

Для построения системы рекомендаций, зная только историю выставленных рейтингов пользователя, будем использовать метод Weighted Approximate-Rank Pairwise ([WARP](http://www.thespermwhale.com/jaseweston/papers/wsabie-ijcai.pdf)). Он реализован в пакете [lightfm](https://github.com/lyst/lightfm).

Чтобы использовать модели, реализованные в пакете, нужно преобразовать исходные данные в разреженные матрицы, с которыми работают модели. Для этого используем класс Dataset из модуля data.

In [10]:
from lightfm.data import Dataset

In [11]:
dataset = Dataset()
dataset.fit((x[0] for x in ratings.itertuples(index=False)),
            (x[1] for x in ratings.itertuples(index=False)))

In [12]:
# Сконструируем разреженную матрицу, состоящую из взаимодействий типа
# пользователь - фильм, используя тренировочный набор данных train_data
# x[0] - 'userId', x[1] - 'movieId'
(train_set, train_weights) = dataset.build_interactions(((x[0], x[1])
                                                          for x in
                                                          train_data.itertuples(index=False)))

In [13]:
# Сконструируем разреженную матрицу, состоящую из взаимодействий типа
# пользователь - фильм, используя тестовый набор данных test_data
# x[0] - 'userId', x[1] - 'movieId'
(test_set, test_weights) = dataset.build_interactions(((x[0], x[1])
                                                        for x in
                                                        test_data.itertuples(index=False)))

In [14]:
model = LightFM(loss='warp', random_state=SEED)
model.fit(train_set, epochs=20)

Поскольку в задаче рекомендации фильмов нам важно, сколько фильмов из тех, которые впоследствии оценил пользователь, мы угадали, для оценки качества модели будем использовать метрику Precision at K. Максимальное значение K, которое соответствует количеству фильмов в списке рекомендаций, выберем равным 10.

In [15]:
precision_at_k(model, test_set, k=10).mean()

0.11581198

Напишем функцию для подбора гиперпараметров модели.

In [16]:
def objective(params):
    '''
    params - list of tuples
    returns average precision at k
    '''
    epochs, learning_rate,\
    no_components, alpha = params
    
    user_alpha = alpha
    item_alpha = alpha
    model = LightFM(loss='warp',
                    random_state=SEED,
                    learning_rate=learning_rate,
                    no_components=no_components,
                    user_alpha=user_alpha,
                    item_alpha=item_alpha)
    model.fit(train_set, epochs=epochs,
              num_threads=4)
    
    patks = precision_at_k(model, test_set,
                           train_interactions=None,
                           k=10)
    apatk = np.mean(patks)
    # Поскольку мы минимизируем значение функции, возвращаем Precision
    # со знаком минус
    out = -apatk
    
    if np.abs(out + 1) < 0.01 or out < -1.0:
        return 0.0
    else:
        return out

In [17]:
space = [(1, 60), # epochs
         (10**-4, 1.0, 'log-uniform'), # learning_rate
         (20, 200), # no_components
         (10**-6, 10**-1, 'log-uniform'), # alpha
        ]

res_fm = forest_minimize(objective, space, n_calls=50,
                         random_state=SEED)

Выберем наилучшие параметры:

In [18]:
print('Maximimum p@k found: {:6.5f}'.format(-res_fm.fun))
params = ['epochs', 'learning_rate', 'no_components', 'alpha']

for (p, x_) in zip(params, res_fm.x):
    print('{}: {}'.format(p, x_))

Maximimum p@k found: 0.12564
epochs: 54
learning_rate: 0.055125073348913936
no_components: 144
alpha: 0.0018668003844694568


In [19]:
modelB = LightFM(loss='warp',
                 learning_rate=0.055,
                 no_components=144,
                 user_alpha=1.9e-03,
                 item_alpha=1.9e-03,
                 random_state=SEED)

start_time = time.time()
modelB.fit(train_set, epochs=54)
train_time = time.time() - start_time
print("Обучение заняло {} секунд.".format(train_time))

Обучение заняло 16.528757572174072 секунд.


In [20]:
precision_at_k(modelB, test_set, k=10).mean()

0.12008548

### Гибридный подход (коллаборативный+контентный)

In [21]:
genres = pd.read_csv('movie.csv')

In [22]:
genres.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [23]:
# Для каждого фильма добавим колонку с жанрами
df2 = ratings.join(genres.set_index('movieId')['genres'], on='movieId', how='left')

In [24]:
tags = pd.read_csv('genome_scores.csv')

In [25]:
tags.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [26]:
# Также возьмем для каждого фильма по три наиболее релевантных тега
rel_tags = tags.groupby('movieId', as_index=False).apply(lambda x: x.nlargest(3, 'relevance')).reset_index(drop=True)

In [27]:
rel_tags.head(9)

,movieId,tagId,relevance
0,1,1036,0.99925
1,1,244,0.99850
2,1,786,0.99600
3,2,29,0.98100
4,2,584,0.96700
5,2,204,0.96425
6,3,451,0.97450
7,3,901,0.95050
8,3,902,0.93325


In [28]:
# Изменим тип тегов на str
rel_tags['tagId'] = rel_tags['tagId'].astype('str')

# Объединим теги для каждого фильма в одну строку
rel_tags = rel_tags.groupby(['movieId'], as_index = False).agg({'tagId': ' '.join})

In [29]:
# Присоединим теги к датафрейму
df2 = df2.join(rel_tags.set_index('movieId')['tagId'], on='movieId', how='left')

In [30]:
df2['genres'] = df2['genres'].apply(lambda x: x.replace('|',' '))

In [31]:
# Для некоторых фильмов отсутствуют пользовательские теги
df2.isna().sum()

userId       0
movieId      0
rating       0
genres       0
tagId      748
dtype: int64

In [32]:
# Заполним пропущенные значения '0'
df2.fillna('0', inplace=True)

In [33]:
# Объединим жанры и теги в одну строчку
df2['genres_tags'] = df2['genres'] + ' ' + df2['tagId']

In [34]:
df2.drop(columns=['genres', 'tagId'], inplace=True)

In [35]:
df2.head()

,userId,movieId,rating,genres_tags
0,1,2,3.5,Adventure Children Fantasy 29 584 204
1,1,29,3.5,Adventure Drama Fantasy Mystery Sci-Fi 287 109...
2,1,32,3.5,Mystery Sci-Fi Thriller 419 1027 1028
3,1,47,3.5,Mystery Thriller 903 300 797
4,1,50,3.5,Crime Mystery Thriller 758 994 789


In [36]:
# Преобразуем столбец genres_tags в разреженную матрицу
countv = CountVectorizer(max_features=25000)
item_features = countv.fit_transform(df2.drop_duplicates(['movieId'])\
                                                         ['genres_tags'])

eye = sp.eye(item_features.shape[0], item_features.shape[0]).tocsr()
item_features_concat = sp.hstack((eye, item_features))
item_features_concat = item_features_concat.tocsr().astype(np.float32)

Немного модернизируем нашу функцию для подбора гиперпараметров модели.

In [37]:
def objective_wsideinfo(params):
    '''
    params - list of tuples
    returns average precision at k
    '''
    epochs, learning_rate,\
    no_components, item_alpha,\
    scale = params
    
    user_alpha = item_alpha * scale
    model = LightFM(loss='warp',
                    random_state=SEED,
                    learning_rate=learning_rate,
                    no_components=no_components,
                    user_alpha=user_alpha,
                    item_alpha=item_alpha)
    model.fit(train_set, epochs=epochs,
              item_features=item_features_concat,
              num_threads=4)
    
    patks = precision_at_k(model, test_set,
                           item_features=item_features_concat,
                           train_interactions=None,
                           k=10)
    apatk = np.mean(patks)
    # Поскольку мы минимизируем значение функции, возвращаем Precision
    # со знаком минус
    out = -apatk

    if np.abs(out + 1) < 0.01 or out < -1.0:
        return 0.0
    else:
        return out

In [38]:
space = [(1, 60), # epochs
         (10**-3, 1.0, 'log-uniform'), # learning_rate
         (20, 200), # no_components
         (10**-5, 10**-3, 'log-uniform'), # item_alpha
         (0.001, 1., 'log-uniform') # user_scaling
        ]

# Начальные значения для параметров возьмем из прошлой оптимизации
# и добавим новое для нового параметра
x0 = res_fm.x.append(1.)

res_fm_itemfeat = forest_minimize(objective_wsideinfo, space, n_calls=50,
                                  x0=x0,
                                  random_state=SEED)

Выберем наилучшие параметры:

In [39]:
print('Maximimum p@k found: {:6.5f}'.format(-res_fm_itemfeat.fun))
params = ['epochs', 'learning_rate', 'no_components', 'item_alpha', 'scaling']

for (p, x_) in zip(params, res_fm_itemfeat.x):
    print('{}: {}'.format(p, x_))

Maximimum p@k found: 0.12821
epochs: 55
learning_rate: 0.005742622693264771
no_components: 127
item_alpha: 0.0008670052220106342
scaling: 0.12350404779287502


In [40]:
modelA = LightFM(loss='warp',
                 learning_rate=5.7e-03,
                 no_components=127,
                 user_alpha=1.0e-04,
                 item_alpha=8.7e-04,
                 random_state=SEED)

start_time = time.time()

modelA.fit(train_set,
           epochs=55,
           item_features=item_features_concat)

train_time = time.time() - start_time
print("Обучение заняло {} секунд.".format(train_time))

Обучение заняло 41.8732373714447 секунд.


In [41]:
precision_at_k(modelA, test_set,
               item_features=item_features_concat, k=10).mean()

0.12507123

Видно, что добавление признаков для фильмов улучшило результат с 0.1201 до 0.1251.

Оценим статистическую значимость результата. Для этого для порога значимости выберем стандартное значение $\alpha = 0.05$ и сформулируем нулевую гипотезу как "Модель с гибридным подходом($A$) работает не лучше, чем модель с коллаборативным подходом($B$)". Предполагая, что набор пользователей в датасете был выбран случайно, мы посчитаем, для скольких пользователей($n_A$) подход $A$ дал результат лучше, чем подход $B$, и для скольких пользователей($n_B$) коллаборативный подход превзошел гибридный. Если алгоритмы одинаково хороши, то $n_A$ и $n_B$ будут равны, поэтому вероятность выбрать пользователя из их суммы $n = n_A + n_B$, для которого один из алгоритмов превзошел другой, будет равна $0.5$. Расчет искомой вероятности производится по формуле:

$p = (0.5)^n \sum_{i=n_A}^{n}\frac{n!}{i!(n-i)!}$

In [42]:
prec_A = precision_at_k(modelA,
                        test_set,
                        item_features=item_features_concat,
                        k=10)

prec_B = precision_at_k(modelB, test_set, k=10)

In [43]:
n_A = np.sum(prec_A > prec_B)
n_B = np.sum(prec_B > prec_A)
n = n_A + n_B

In [44]:
summ = 0
for i in range(n_A, n+1):
    summ += (math.factorial(n) /
            (math.factorial(i) * math.factorial(n-i)))

In [45]:
p = ((0.5)**n) * summ

In [46]:
p

0.2005313997960155

Вероятность превышает выбранный порог 0.05, поэтому можем сказать, что нулевая гипотеза выполняется, и гибридный подход в нашем случае сработал не лучше, чем коллаборативный.